In [20]:
#!/usr/bin/env python
import pandas as pd
from python_speech_features import mfcc
from python_speech_features import delta
from python_speech_features import logfbank
import scipy.io.wavfile as wav
import glob
import scipy.io
import librosa
import numpy as np

In [34]:
def average(data,window_size):
    df = pd.DataFrame()
    for i in range(int(len(data)/window_size)):
        dt = np.array(data[i*window_size:(i+1)*window_size])
        window = np.ones((window_size,))/float(window_size)
        dt = pd.DataFrame(np.convolve(dt, window,mode='valid'))
        df = pd.concat([df,dt])
    return df

In [35]:
def root_mean_square(data,window_size):
    df = pd.DataFrame()
    for i in range(int(len(data)/window_size)):
        dt = np.power(data[i*window_size:(i+1)*window_size],2)
        window = np.ones((window_size,))/float(window_size)
        dt = pd.DataFrame(np.sqrt(np.convolve(dt, window,mode='valid')))
        df = pd.concat([df,dt])
    return df

In [24]:
#MFCC_Proccessing
path ='/home/guatam-admin/Vijay_BCI/Data-Mason-1/Audio wav files' # use your path
allFiles = glob.glob(path + "/*.wav")
for file_ in allFiles:
    (rate,sig) = wav.read(file_)
    mfcc_feat = mfcc(sig,rate,winstep=0.01)
    mfcc_feat = pd.DataFrame(mfcc_feat)
    mfcc_feat.to_csv(file_[:-4]+'.csv',index= False)

In [25]:
#Convert .mat to .csv
path ='/home/guatam-admin/Vijay_BCI/Data-Mason-1/EEG .mat files' # use your path
allFiles = glob.glob(path + "/*.mat")
for file_ in allFiles:
    mat = scipy.io.loadmat(file_)
    mat = {k:v for k, v in mat.items() if k[0] != '_'}
    names = list(mat.keys())
    df = pd.DataFrame()
    for key in names[:-8]:
        value = mat[key]
        value = pd.DataFrame(value)
        df = pd.merge(df,value,how='right',right_index=True,left_index=True)
    df.columns = names[:-8]
    df.to_csv(file_[:-4]+'.csv',index=False)

In [32]:
#Zero_crossing_rate
path ='/home/guatam-admin/Vijay_BCI/Data-Mason-1/EEG .csv files' # use your path
allFiles = glob.glob(path + "/*.csv")
for file_ in allFiles:
    df = pd.read_csv(file_)
    df_zero = pd.DataFrame()
    for name in df.columns:
        zero = librosa.feature.zero_crossing_rate(df[name].values,frame_length=10,hop_length=10)
        zero = pd.DataFrame(np.transpose(zero))
        df_zero = pd.merge(df_zero,zero,how='right',right_index=True,left_index=True)
    df_zero.columns = df.columns
    df_zero = df_zero*100
    df_zero.to_csv(file_[:-4]+'_zero.csv',index=False)

In [ ]:
#Average
#Do for each file
path ='/home/guatam-admin/Vijay_BCI/Data-Mason-1/EEG .csv files/Mason-filters-e.csv' # use your path
df_avg = pd.DataFrame()
for name in df.columns:
    avg = average(df[name].values,window_size=10)
    #rms = pd.DataFrame(np.transpose(zero))
    df_avg = pd.concat([df_avg,avg],axis=1)
df_avg.columns = df.columns
#df_rms = df_zero*100
df_avg.to_csv('/home/guatam-admin/Vijay_BCI/Data-Mason-1/EEG_average/Mason-e-avg.csv',index=False)

In [9]:
#Root_mean_square
#Do for each file
path ='/home/guatam-admin/Vijay_BCI/Data-Mason-1/EEG .csv files' # use your path
df_rms = pd.DataFrame()
for name in df.columns:
    rms = root_mean_square(df[name].values,window_size=100)
    #rms = pd.DataFrame(np.transpose(zero))
    df_rms = pd.concat([df_rms,rms],axis=1)
df_rms.columns = df.columns
#df_rms = df_zero*100
df_rms.to_csv('/home/guatam-admin/Vijay_BCI/Data/data-u_rms.csv',index=False)

In [ ]:
#Trimmming - get the lengths of audio files
path ='/home/guatam-admin/Vijay_BCI/Data/Audio_csv' # use your path
allFiles = glob.glob(path + "/*.csv")
length = []
for file_ in allFiles:
    length.append(len(pd.read_csv(file_)))

In [ ]:
#Matching the length with the .csv files
#Do for each file
path_ ='/home/guatam-admin/Vijay_BCI/Data/Average' # use your path
allFiles_ = glob.glob(path_ + "/*.csv")
for file_ in allFiles_:
    for file in allFiles:
        if(file[-5]==file_[-5]):
            df = pd.read_csv(file_)
            df = df[(len(df)-len(pd.read_csv(file))):len(df)]
            df.to_csv(file_[:-4]+'_trimmed.csv',index= False)
    #print(i,file_)